In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output


In [ ]:
#  load nbaPlayersAllTimesAgilityData.csv, nbaPlayersAllTimesInfoData and nbaPlayersAllTimesShootingData.csv
dfAg = pd.read_csv('nbaPlayersAllTimesAgilityData.csv')
dfIn = pd.read_csv('nbaPlayersAllTimesInfo.csv')
dfStats = pd.read_csv('nbaPlayersAllTimesStatsData.csv')


#  remove ' ' from column names /!\ invisible characters
dfAg.columns = dfAg.columns.str.replace(' ', ' ')
dfIn.columns = dfIn.columns.str.replace(' ', ' ')
dfStats.columns = dfStats.columns.str.replace(' ', ' ')

# remane colum Player to PLAYER
dfIn = dfIn.rename(columns={'Player': 'PLAYER'})

# remove space in playerName
dfAg['PLAYER'] = dfAg['PLAYER'].str.replace(' ', '')
dfIn['PLAYER'] = dfIn['PLAYER'].str.replace(' ', '')
dfStats['PLAYER'] = dfStats['PLAYER'].str.replace(' ', '')

# remove duplicates
dfAg = dfAg.drop_duplicates(subset=['PLAYER'], keep='first')
dfIn = dfIn.drop_duplicates(subset=['PLAYER'], keep='first')
dfStats = dfStats.drop_duplicates(subset=['PLAYER'], keep='first')

# add the 3 dataframes
df = pd.merge(dfAg, dfIn, on='PLAYER', how='outer')
df = pd.merge(df, dfStats, on='PLAYER', how='outer')

# replace every value egal to '-' with the mean of the other value in the columns
df = df.replace('-', np.nan)
df = df.apply(pd.to_numeric, errors='ignore')
df = df.fillna(df.mean(numeric_only=True))

# convert player name to string
df['PLAYER'] = df['PLAYER'].astype(str)



# convert inch to meters and round to decimal
df['Max Vertical Leap (inches)'] = df['Max Vertical Leap (inches)'].apply(pd.to_numeric)
df['Max Vertical Leap (meters)'] = round(df['Max Vertical Leap (inches)'] * 0.0254, 2)

# convert inch to meters and round to decimal
df['Standing Vertical Leap (inches)'] = df['Standing Vertical Leap (inches)'].apply(pd.to_numeric)
df['Standing Vertical Leap (meters)'] = round(df['Standing Vertical Leap (inches)'] * 0.0254, 2)

# remove inch columns
df = df.drop(['Max Vertical Leap (inches)', 'Standing Vertical Leap (inches)'], axis=1)

# convert height to meters and round to decimal
df['Height'] = df['Height'].apply(pd.to_numeric)
df['Height'] = round(df['Height'] / 100, 2)

print(df.shape)

# df1 = to df minus none numeric columns
df1 = df.select_dtypes(exclude=['object'])
df1 = df1 * 1000
print(df1.shape)

# print correlation matrix
corr = df1.corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corr, annot=True, fmt='.2f', vmin=-1, vmax=1, center=0, cmap='coolwarm', square=True)
plt.show()

# But
Faire un spider plot des compétences du joueur choisi par l'utilisateur

In [ ]:
#name = input("Enter the name of the player : ").lower().replace(' ', '')
#playersName = df['PLAYER'].map(str.lower)

#while(name not in playersName.values):
#   print("Player ", name ," not found")
 #   name = input("Enter the name of the player : ").lower().replace(' ', '')

clear_output()
name="lebronjames"
print("Begin analysis of ", name)

print(df[df['PLAYER'].str.lower() == name])
# print df size
print("df size : ", df.shape)
#chosen player
player = df[df['PLAYER'].str.lower() == name]

# display player info as a radar chart
categories = list(player)[1:]
N = len(categories)
